# Using the SequencerUtils

In [1]:
# import pylon and the sequencer utils
from pypylon import pylon as py
from utils.sequencer.sequencer import CameraSequence, SinglePathSet, SequencerTransition

In [2]:
# create a camera instance an open it
my_cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice())
my_cam.Open()

# create a sequence object with the camera
# this will be used to configure the sequencer
sequence = CameraSequence(camera=my_cam)

#### Define the transitions to change the sequencer to the next set


In [3]:
# if a software signal is triggerd (manual control)
software_transition = SequencerTransition(trigger_source="SoftwareSignal1", trigger_activation="NA")
# if an exposure is started (change every frame)
next_frame_transition = SequencerTransition(trigger_source="ExposureStart", trigger_activation="NA")

###  Setup three sets
#### Remain in the "Medium" set until a software signal occur, than do a set with a "Low", and a set with a "High" exposure time
#### Use "auto_close_loop" to return to the "Medium" set after the "High" set without explicitly setting the next set in the "High" set.


In [4]:
# appending the sets to the sequence will define the order of transition, if next_set_number is not explicitly set in the SinglePathSet
sequence.append(SinglePathSet(name="Medium", set_number=0, feature_set={"ExposureTime": 1000}, transition=software_transition))
sequence.append(SinglePathSet(name="Low", set_number=1, feature_set={"ExposureTime": 100}, transition=next_frame_transition))
sequence.append(SinglePathSet(name="High", set_number=2, feature_set={"ExposureTime": 5000}, transition=next_frame_transition))

# By using auto close loop, we return into the "Medium" Set after leaving set: "High" automatically.
# This could also be done by setting the next set in the SetConstructor, like:
# SinglePathSet(name="High",set_number=2,next_set_number=0,feature_set={"ExposureTime": 5000}, transition=next_frame_transition)
sequence.configure(auto_close_loop=True)

In [5]:
# The sequence supports a string representation, which can be used to print the sequence in a human readable format.
# This is useful for debugging and understanding the sequence flow.
print("Sequence Configuration:")
for i in sequence:
    print(f"\t{i}")

Sequence Configuration:
	[0] (Name: Medium) switch to [1] if Event on SoftwareSignal1
	[1] (Name: Low) switch to [2] if Event on ExposureStart
	[2] (Name: High) switch to [0] if Event on ExposureStart


In [6]:
sequence.activate()

In [7]:
# close the camera to allow access in the pylon viewer
my_cam.Close()

The sequence is now configured and activated. You can now start the camera and it will automatically switch between the sets based on the defined transitions.
You can easyly verify this by using the pylon viewer and start continuous acquisition. By using the software signal,
the camera will change from Medium to Low and High exposure time, and return to Medium after the High set, waiting for the next software signal to change to Low again.
